In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import multiprocessing as mp
import utils as utils
import time

In [ ]:
mp.cpu_count()

In [ ]:
tic = time.time()
p = mp.Pool(mp.cpu_count())    
p.map(utils.f, range(10));
print(time.time()-tic)

In [ ]:
tic = time.time()
[utils.f(i) for i in range(10)]
print(time.time()-tic)

In [1]:
# Code to accompany the paper "Constructions in combinatorics via neural networks and LP solvers" by A Z Wagner
# Code for conjecture 2.1, without the use of numba 
#
# Please keep in mind that I am far from being an expert in reinforcement learning. 
# If you know what you are doing, you might be better off writing your own code.
#
# This code works on tensorflow version 1.14.0 and python version 3.6.3
# It mysteriously breaks on other versions of python.
# For later versions of tensorflow there seems to be a massive overhead in the predict function for some reason, and/or it produces mysterious errors.
# Debugging these was way above my skill level.
# If the code doesn't work, make sure you are using these versions of tf and python.
# I used keras version 2.3.1, not sure if this is important, but I recommend this just to be safe.

import networkx as nx #for various graph parameters, such as eigenvalues, macthing number, etc
import random
import numpy as np
import copy
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.models import load_model
from statistics import mean
from math import sqrt
from numpy.random import choice
import pickle
import time
import math
import sympy
import matplotlib.pyplot as plt

In [2]:
from stabilizer_search.search.brute_force import *
from stabilizer_search.mat import X, T
from stabilizer_search.mat import tensor
import randstab as rs

In [3]:
n_qubits = 4
chi = 6
H = [[np.cos(np.pi/8)],[np.sin(np.pi/8)]]
target_state = tensor(*([H]*n_qubits))

In [4]:
# Daochen: is there a way of knowing the number of real stabilizer states? O randomly generating real 
print('number of all stabilizer states =', sum(rs.number_of_states(n_qubits)))
n_stabilizers_target = 20000

number of all stabilizer states = 36720


In [5]:
stabilizers = [rs.random_stabilizer_state(n_qubits) for i in range(n_stabilizers_target)]
L = {array.tobytes(): array for array in stabilizers}
unique_stabilizers = L.values() # [array([1, 3, 2, 4]), array([1, 2, 3, 4])]
unique_real_stabilizers = list(filter(lambda x: all(np.isreal(x)), list(unique_stabilizers)))
stabilizers = np.array(unique_real_stabilizers)
n_stabilizers = len(stabilizers)
print('number of considered stabilizer states =', n_stabilizers)

number of considered stabilizer states = 2019


In [6]:
# sanity check that the target is in the span
candidate_stabilizer_basis = [np.array([stabilizers[i,:]]).transpose() for i in range(n_stabilizers)]
projector = ortho_projector(candidate_stabilizer_basis)
projection = np.linalg.norm(projector*target_state, 2)

In [7]:
projection

1.0

In [ ]:
# DON'T DO THIS! E.g. say you had |0>, |1>, |+> and you killed off |+> because it's linearly dependent, then you get a bad stabilizer decomposition of |+>!
# _, inds = sympy.Matrix(candidate_stabilizer_states).T.rref()
# candidate_stabilizer_states = candidate_stabilizer_states[inds,:]
# n_candidates_actual = candidate_stabilizer_states.shape[0]

In [8]:
# Daochen: would be easier if could enforce the use of combinations
# MYN = int(2**n)  #The length of the word we are generating. Here we are generating a Boolean function on n bits, so we create a 0-1 word of length 2^n

MYN = int(chi)

# LEARNING_RATE = 0.0001 #Increase this to make convergence faster, decrease if the algorithm gets stuck in local optima too often.
LEARNING_RATE = 0.00001
n_sessions = 1000 #number of new sessions per iteration
# default 93, 94 respectively
percentile = 93 #top 100-X percentiled we are learning from
super_percentile = 97 #top 100-X percentile that survives to next iteration

# These are hyperparameters
FIRST_LAYER_NEURONS = 128 #Number of neurons in the hidden layers.
SECOND_LAYER_NEURONS = 64
THIRD_LAYER_NEURONS = 32

# n_actions = 2
# Daochen: note that this parameter is not actually used anywhere.
n_actions = n_stabilizers
#The size of the alphabet. In this file we will assume this is 2. There are a few things we need to change when the alphabet size is larger,
#such as one-hot encoding the input, and using categorical_crossentropy as a loss function.

observation_space = 2*MYN 

# Leave this at 2*MYN. The input vector will have size 2*MYN, 
# where the first MYN letters encode our partial word (with zeros on
# the positions we haven't considered yet), and the next MYN bits one-hot encode which letter we are considering now.
# So e.g. [0,1,0,0,   0,0,1,0] means we have the partial word 01 and we are considering the third letter now.
# Is there a better way to format the input to make it easier for the neural network to understand things?

# Daochen: why should len_game have anything to do with MYN
len_game = MYN 
state_dim = (observation_space,)

INF = 1000000

#Model structure: a sequential network with three hidden layers, sigmoid activation in the output.
#I usually used relu activation in the hidden layers but play around to see what activation function and what optimizer works best.
#It is important that the loss is binary cross-entropy if alphabet size is 2.

model = Sequential()
model.add(Dense(FIRST_LAYER_NEURONS,  activation="relu"))
model.add(Dense(SECOND_LAYER_NEURONS, activation="relu"))
model.add(Dense(THIRD_LAYER_NEURONS, activation="relu"))
model.add(Dense(n_stabilizers, activation="softmax"))
model.build((None, observation_space))
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate = LEARNING_RATE)) #Adam optimizer also works well, with lower learning rate

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1664      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2019)              66627     
Total params: 78,627
Trainable params: 78,627
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# candidate_stabilizer_basis = [np.array([candidate_stabilizer_states[i,:]]).transpose() for i in range(MYN)];

In [9]:
def calcScore(state):
    """
    Calculates the reward for a given word. 
    This function is very slow, it can be massively sped up with numba -- but numba doesn't support networkx yet, which is very convenient to use here
    :param state: the first MYN letters of this param are the word that the neural network has constructed.

    :returns: the reward (a real number). Higher is better, the network will try to maximize this.
    """

    f = state[:MYN];
    candidate_stabilizer_basis = [np.array([stabilizers[f[i],:]]).transpose() for i in range(MYN)]
#     print(candidate_stabilizer_basis)
    projector = ortho_projector(candidate_stabilizer_basis)
    projection = np.linalg.norm(projector*target_state, 2)
#     projection = 1
    
    score = projection
    target = score
    
    if np.allclose(score, 1):
        print('You found a stabilizer decomposition with (n_qubits,chi) = ', [n_qubits,chi])
        print('The set of stabilizers is: ', f)
        return -1, -1
    return target, score

####No need to change anything below here. 
# Daochen: the agent argument will be the "model"
def generate_session(agent, n_sessions, verbose = 1):
    """
    Play n_session games using agent neural network.
    Terminate when games finish 

    Code inspired by https://github.com/yandexdataschool/Practical_RL/blob/master/week01_intro/deep_crossentropy_method.ipynb
    """
    states =  np.zeros([n_sessions, observation_space, len_game], dtype=int)
    actions = np.zeros([n_sessions, len_game], dtype = int)
    state_next = np.zeros([n_sessions,observation_space], dtype = int)
    prob = np.zeros(n_sessions)
    states[:,MYN,0] = 1
    step = 0
    total_target = np.zeros([n_sessions])
#     total_target = np.zeros([n_sessions], dtype=complex)
    total_score = np.zeros([n_sessions])
    recordsess_time = 0
    play_time = 0
    scorecalc_time = 0
    pred_time = 0
    while (True):
        step += 1
        tic = time.time()
        prob = agent.predict(states[:,:,step-1], batch_size = n_sessions) 
        pred_time += time.time()-tic

        for i in range(n_sessions):
            action = choice(n_stabilizers, p=prob[i])
            actions[i][step-1] = action
            tic = time.time()
            state_next[i] = states[i,:,step-1]
            play_time += time.time()-tic
            if (action > 0):
                state_next[i][step-1] = action
            state_next[i][MYN + step-1] = 0
            if (step < MYN):
                state_next[i][MYN + step] = 1
#                 Daochen: terminal equals whether step equals MYN: I suppose meaning that an entire state has been generated
            terminal = step == MYN
            tic = time.time()
            if terminal:
#                 print('state_next[i]', state_next[i])
                total_target[i], total_score[i] = calcScore(state_next[i])
                if total_target[i] == -1:
                    return -1
#                 print("total_score", total_score[i])
            scorecalc_time += time.time()-tic
            tic = time.time()
            if not terminal:
                states[i,:,step] = state_next[i]
            recordsess_time += time.time()-tic
        if terminal:
            break
    #If you want, print out how much time each step has taken. This is useful to find the bottleneck in the program.		
    if (verbose):
        print("Predict: "+str(pred_time)+", play: " + str(play_time) +", scorecalc: " + str(scorecalc_time) +", recordsess: " + str(recordsess_time))
    return states, actions, total_score, total_target

def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    This function was mostly taken from https://github.com/yandexdataschool/Practical_RL/blob/master/week01_intro/deep_crossentropy_method.ipynb
    If this function is the bottleneck, it can easily be sped up using numba
    """
    counter = n_sessions * (100.0 - percentile) / 100.0
    reward_threshold = np.percentile(rewards_batch,percentile)

    elite_states = []
    elite_actions = []
    elite_rewards = []
    for i in range(len(states_batch)):
        if rewards_batch[i] >= reward_threshold-0.0000001:
            if (counter > 0) or (rewards_batch[i] >= reward_threshold+0.0000001):
                for item in states_batch[i]:
                    elite_states.append(item.tolist())
                for item in actions_batch[i]:
                    elite_actions.append(item)
            counter -= 1
    elite_states = np.array(elite_states, dtype = int)
    elite_actions = np.array(elite_actions, dtype = int)
    return elite_states, elite_actions

def select_super_sessions(states_batch, actions_batch, rewards_batch, targets_batch, percentile=90):
    """
    Select all the sessions that will survive to the next generation
    Similar to select_elites function
    If this function is the bottleneck, it can easily be sped up using numba
    """
    counter = n_sessions * (100.0 - percentile) / 100.0
    reward_threshold = np.percentile(rewards_batch,percentile)

    super_states = []
    super_actions = []
    super_rewards = []
    super_targets = []
    for i in range(len(states_batch)):
        if rewards_batch[i] >= reward_threshold-0.0000001:
            if (counter > 0) or (rewards_batch[i] >= reward_threshold+0.0000001):
                super_states.append(states_batch[i])
                super_actions.append(actions_batch[i])
                super_rewards.append(rewards_batch[i])
                super_targets.append(targets_batch[i])
                counter -= 1
    super_states = np.array(super_states, dtype = int)
    super_actions = np.array(super_actions, dtype = int)
    super_rewards = np.array(super_rewards)
    super_targets = np.array(super_targets)
    return super_states, super_actions, super_rewards, super_targets

In [10]:
super_states =  np.empty((0,len_game,observation_space), dtype = int)
super_actions = np.array([], dtype = int)
super_rewards = np.array([])
super_targets= np.array([])
sessgen_time = 0
fit_time = 0
score_time = 0

myRand = random.randint(0,1000) #used in the filename
#index = 0 #used to index generation

In [16]:
for i in range(5000): ### for i in range(5000): #1000000 generations should be plenty
    #generate new sessions
    #performance can be improved with joblib
    tic = time.time()
#     sessions = states, actions, total_score, total_target
    sessions = generate_session(model,n_sessions,0) #change 0 to 1 to print out how much time each step in generate_session takes 
    if sessions == -1:
        break
    sessgen_time = time.time()-tic
    tic = time.time()

    states_batch = np.array(sessions[0], dtype = int)
    actions_batch = np.array(sessions[1], dtype = int)
    rewards_batch = np.array(sessions[2])
    targets_batch = np.array(sessions[3])
    
    states_batch = np.transpose(states_batch,axes=[0,2,1])
    states_batch = np.append(states_batch,super_states,axis=0)

    if i>0:
        actions_batch = np.append(actions_batch,np.array(super_actions),axis=0)	
    
    rewards_batch = np.append(rewards_batch,super_rewards)
    targets_batch = np.append(targets_batch,super_targets)

    randomcomp_time = time.time()-tic 
    tic = time.time()

    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile=percentile) #pick the sessions to learn from
    select1_time = time.time()-tic

    tic = time.time()
    super_sessions = select_super_sessions(states_batch, actions_batch, rewards_batch, targets_batch, percentile=super_percentile) #pick the sessions to survive
    select2_time = time.time()-tic

    tic = time.time()
    super_sessions = [(super_sessions[0][i], super_sessions[1][i], super_sessions[2][i], super_sessions[3][i]) for i in range(len(super_sessions[2]))]
    super_sessions.sort(key=lambda super_sessions: super_sessions[2],reverse=True)
    select3_time = time.time()-tic

    tic = time.time()
    model.fit(elite_states, elite_actions, verbose=0) #learn from the elite sessions
    fit_time = time.time()-tic

    tic = time.time()

    super_states = [super_sessions[i][0] for i in range(len(super_sessions))]
    super_actions = [super_sessions[i][1] for i in range(len(super_sessions))]
    super_rewards = [super_sessions[i][2] for i in range(len(super_sessions))]
    super_targets = [super_sessions[i][3] for i in range(len(super_sessions))]
    
    #     print(super_states)

    rewards_batch.sort()
#     Daochen: why is it -100?
    mean_all_reward = np.mean(rewards_batch[-100:])
    mean_best_reward = np.mean(super_rewards)

    score_time = time.time()-tic

    
#Aarthi: Formatting output for logs and display. PLEASE DON'T DELETE CODE. Comment it out if not needed so it doesn't get lost and need to be re-written each time
    print("\n" + str(i) +  ". Best individuals (reward): " + str(np.flip(np.sort(super_rewards))))
#     Daochen: note that sometimes it makes sense to add/remove np.flip below
    print("\n" + str(i) +  ". Best individuals (target): " + str(np.flip(np.sort(super_targets))))

    #uncomment below line to print out how much time each step in this loop takes. 
    print("Mean reward: " + str(mean_all_reward) + "\nSessgen: " + str(sessgen_time) + ", other: " + str(randomcomp_time) + ", select1: " + str(select1_time) + ", select2: " + str(select2_time) + ", select3: " + str(select3_time) +  ", fit: " + str(fit_time) + ", score: " + str(score_time))
    
    with open('Run'+str(myRand)+'.txt', 'a') as f:
        f.write("\n" + str(i) +  ". Best individuals (reward): " + str(np.flip(np.sort(super_rewards))))
        f.write("\n" + str(i) +  ". Best individuals (target): " + str(np.sort(super_targets)))
        f.write("Mean reward: " + str(mean_all_reward) + "\nSessgen: " + str(sessgen_time) + ", other: " + str(randomcomp_time) + ", select1: " + str(select1_time) + ", select2: " + str(select2_time) + ", select3: " + str(select3_time) +  ", fit: " + str(fit_time) + ", score: " + str(score_time)+"\n")

    if (i%20 == 1): #Write all important info to files every 20 iterations
#         with open('best_species_pickle_'+str(myRand)+'.txt', 'wb') as fp:
#             pickle.dump(super_actions, fp)
        with open('super_actions_'+str(myRand)+'.txt', 'a') as f:
            f.write("At i = "+str(i)+":"+"\n")
            for item in super_actions:
                f.write(str(item))
                f.write("\n")
        with open('super_rewards_'+str(myRand)+'.txt', 'a') as f:
            f.write("At i = "+str(i)+":"+"\n")
            for item in super_rewards:
                f.write(str(item))
                f.write("\n")
        with open('mean_rewards_'+str(myRand)+'.txt', 'a') as f:
             f.write("At i = "+str(i)+": "+ str(mean_all_reward)+"\n")
        with open('mean_best_rewards_'+str(myRand)+'.txt', 'a') as f:
             f.write("At i = "+str(i)+": "+ str(mean_best_reward)+"\n")


0. Best individuals (reward): [0.86664131 0.8606064  0.85389386 0.82860969 0.82589763 0.81953359
 0.81803485 0.81250337 0.80998981 0.80259798 0.80248334 0.80100004
 0.7989121  0.79242365 0.78766065 0.78656132 0.78550212 0.78406803
 0.78343223 0.78176355 0.78171295 0.78141238 0.78107924 0.77781793
 0.77557408 0.77433876 0.77361212 0.77268571 0.77017592 0.76965651]

0. Best individuals (target): [0.86664131 0.8606064  0.85389386 0.82860969 0.82589763 0.81953359
 0.81803485 0.81250337 0.80998981 0.80259798 0.80248334 0.80100004
 0.7989121  0.79242365 0.78766065 0.78656132 0.78550212 0.78406803
 0.78343223 0.78176355 0.78171295 0.78141238 0.78107924 0.77781793
 0.77557408 0.77433876 0.77361212 0.77268571 0.77017592 0.76965651]
Mean reward: 0.7451684810253795
Sessgen: 3.846621513366699, other: 0.0, select1: 0.0010128021240234375, select2: 0.0009949207305908203, select3: 0.0, fit: 0.46498942375183105, score: 0.0

1. Best individuals (reward): [0.91592492 0.86664131 0.86161653 0.8606064  0.8


9. Best individuals (reward): [0.94593555 0.94577876 0.93342755 0.91592492 0.9123176  0.91216415
 0.90843798 0.90835008 0.903859   0.90189979 0.89588033 0.89354318
 0.89330398 0.89214882 0.88584012 0.88446361 0.88431384 0.88401283
 0.8827356  0.88076471 0.87697898 0.87485835 0.87445484 0.87437881
 0.87320342 0.87296723 0.87278079 0.87146715 0.87076807 0.86972003
 0.86956452]

9. Best individuals (target): [0.94593555 0.94577876 0.93342755 0.91592492 0.9123176  0.91216415
 0.90843798 0.90835008 0.903859   0.90189979 0.89588033 0.89354318
 0.89330398 0.89214882 0.88584012 0.88446361 0.88431384 0.88401283
 0.8827356  0.88076471 0.87697898 0.87485835 0.87445484 0.87437881
 0.87320342 0.87296723 0.87278079 0.87146715 0.87076807 0.86972003
 0.86956452]
Mean reward: 0.8418359869590732
Sessgen: 3.515061378479004, other: 0.0, select1: 0.0020220279693603516, select2: 0.0, select3: 0.0009920597076416016, fit: 0.07395339012145996, score: 0.0

10. Best individuals (reward): [0.94593555 0.94577876 


18. Best individuals (reward): [0.94617635 0.94593555 0.94577876 0.93588849 0.93547527 0.93342755
 0.93029415 0.9289404  0.92608743 0.92307519 0.92268256 0.9219858
 0.92126262 0.91988318 0.917861   0.91592492 0.9149293  0.91364136
 0.91332827 0.9123176  0.91219937 0.91216415 0.91093902 0.90965986
 0.9086131  0.90861263 0.90843798 0.90835008 0.90793078 0.90784055
 0.90684058]

18. Best individuals (target): [0.94617635 0.94593555 0.94577876 0.93588849 0.93547527 0.93342755
 0.93029415 0.9289404  0.92608743 0.92307519 0.92268256 0.9219858
 0.92126262 0.91988318 0.917861   0.91592492 0.9149293  0.91364136
 0.91332827 0.9123176  0.91219937 0.91216415 0.91093902 0.90965986
 0.9086131  0.90861263 0.90843798 0.90835008 0.90793078 0.90784055
 0.90684058]
Mean reward: 0.8831899816938292
Sessgen: 5.215805530548096, other: 0.0, select1: 0.0019986629486083984, select2: 0.00099945068359375, select3: 0.0, fit: 0.07759952545166016, score: 0.0

19. Best individuals (reward): [0.94870793 0.94617635 0.


27. Best individuals (reward): [0.96171    0.96162081 0.95643689 0.95643689 0.94870793 0.94617635
 0.94593555 0.94577876 0.94566558 0.94239157 0.94218417 0.9412691
 0.93979577 0.93944191 0.93877623 0.93600124 0.93588849 0.93547527
 0.93380267 0.93342755 0.93304326 0.93029415 0.93002311 0.93000461
 0.92930947 0.9289404  0.92889128 0.92864293 0.92830799 0.92757643
 0.92608743]

27. Best individuals (target): [0.96171    0.96162081 0.95643689 0.95643689 0.94870793 0.94617635
 0.94593555 0.94577876 0.94566558 0.94239157 0.94218417 0.9412691
 0.93979577 0.93944191 0.93877623 0.93600124 0.93588849 0.93547527
 0.93380267 0.93342755 0.93304326 0.93029415 0.93002311 0.93000461
 0.92930947 0.9289404  0.92889128 0.92864293 0.92830799 0.92757643
 0.92608743]
Mean reward: 0.9085648190517243
Sessgen: 7.473052024841309, other: 0.0009996891021728516, select1: 0.0029997825622558594, select2: 0.0009996891021728516, select3: 0.0010001659393310547, fit: 0.12200307846069336, score: 0.0010039806365966797




36. Best individuals (reward): [0.9954715  0.96171    0.96162081 0.96162081 0.96162081 0.96162081
 0.95881894 0.95881894 0.95643689 0.95643689 0.95643689 0.95583958
 0.95583958 0.95528241 0.95298722 0.95100553 0.94870793 0.94739495
 0.94702475 0.94617635 0.94593555 0.94577876 0.94566558 0.94553685
 0.94548941 0.94473831 0.94471049 0.9438987  0.94363703 0.94286881
 0.94265579]

36. Best individuals (target): [0.9954715  0.96171    0.96162081 0.96162081 0.96162081 0.96162081
 0.95881894 0.95881894 0.95643689 0.95643689 0.95643689 0.95583958
 0.95583958 0.95528241 0.95298722 0.95100553 0.94870793 0.94739495
 0.94702475 0.94617635 0.94593555 0.94577876 0.94566558 0.94553685
 0.94548941 0.94473831 0.94471049 0.9438987  0.94363703 0.94286881
 0.94265579]
Mean reward: 0.9226981398304881
Sessgen: 4.744326829910278, other: 0.0, select1: 0.002998828887939453, select2: 0.0009999275207519531, select3: 0.0, fit: 0.08801007270812988, score: 0.0

37. Best individuals (reward): [0.9954715  0.96171   

...


1895. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]

1895. Best individuals (target): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]
Mean reward: 0.9962109417349565
Sessgen: 4.1268393993377686, other: 0.0010075569152832031, select1: 0.0020029544830322266, select2: 0.0009999275207519531, select3: 0.0, fit: 0.07499551773071289, score: 0.0

1896. Best individuals (reward): [0.99621094 0.996210


1904. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]

1904. Best individuals (target): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]
Mean reward: 0.9962109417349565
Sessgen: 3.824449300765991, other: 0.0, select1: 0.004000425338745117, select2: 0.0010018348693847656, select3: 0.0, fit: 0.07388710975646973, score: 0.0

1905. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.9962


1913. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]

1913. Best individuals (target): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]
Mean reward: 0.9962109417349565
Sessgen: 3.8225057125091553, other: 0.0, select1: 0.0020012855529785156, select2: 0.000949859619140625, select3: 0.0, fit: 0.07709121704101562, score: 0.0

1914. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.996


1922. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]

1922. Best individuals (target): [0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094
 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094 0.99621094]
Mean reward: 0.9962109417349565
Sessgen: 4.876385450363159, other: 0.0, select1: 0.0029697418212890625, select2: 0.0010302066802978516, select3: 0.0, fit: 0.09197139739990234, score: 0.0

1923. Best individuals (reward): [0.99621094 0.99621094 0.99621094 0.996

KeyboardInterrupt: 